In [90]:
import json
import pandas as pd

# Load the JSON data from the file
with open('exercises.json', 'r') as f:
    data = json.load(f)

name, force, level, equipment, primary_muscles, secondary_muscles, category = [], [], [], [], [], [], []
equipment_filter = ['barbell', 'dumbbell', 'machine', "e-z curl bar", "cable", "body only"]

# Iterate over each muscle group and each exercise within that group
for muscle_group in data:
    for exercise in data[muscle_group]:
        if exercise.get('equipment') in equipment_filter:
            name.append(exercise['name'])
            level.append(exercise.get('level', 'N/A'))  # Assuming 'level' corresponds to 'difficulty'
            equipment.append(exercise.get('equipment', 'N/A'))
            primary_muscles.append(exercise.get('primaryMuscles'))
            secondary_muscles.append(exercise.get('secondaryMuscles'))
            category.append(exercise.get('category'))

# Create a DataFrame
df = pd.DataFrame({'name': name, 'level': level, 'equipment': equipment, 'primary_muscles': primary_muscles, 'secondary_muscles': secondary_muscles, 'category': category})
df.head()

,name,level,equipment,primary_muscles,secondary_muscles,category
0,3/4 Sit-Up,beginner,body only,[abdominals],[],strength
1,90/90 Hamstring,beginner,body only,[hamstrings],[calves],stretching
2,Ab Crunch Machine,intermediate,machine,[abdominals],[],strength
3,Adductor/Groin,intermediate,body only,[adductors],[],stretching
4,Air Bike,beginner,body only,[abdominals],[],strength


In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df['features'] = df.apply(lambda x: ' '.join(x['primary_muscles']) + ' ' + x['category'], axis=1)

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the features
tfidf_matrix = vectorizer.fit_transform(df['features'])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def get_recommendations(name, cosine_sim=cosine_sim):
    # Get the index of the exercise
    idx = df[df['name'] == name].index[0]

    # Get the pairwise similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the exercises based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar exercises
    sim_scores = sim_scores[1:6]

    # Get the exercise indices
    exercise_indices = [i[0] for i in sim_scores]

    # Return the top 5 most similar exercises
    return df['name'].iloc[exercise_indices]

# Example usage
print("Recommendations for 'Pullups':")
print(get_recommendations('Pullups'))

Recommendations for 'Pullups':
80                Cable Incline Pushdown
110                              Chin-Up
125        Close-Grip Front Lat Pulldown
195                  Elevated Cable Rows
223    Full Range-Of-Motion Lat Pulldown
Name: name, dtype: object
